![image](images/um_logo.png)

# Guía Completa de `asyncio` en Python

Se proporciona una guía detallada sobre cómo usar `asyncio` en Python. `asyncio` es un módulo que permite programar tareas de manera asíncrona y concurrente usando coroutines. Este documento es útil para estudiantes que buscan comprender y aplicar asincronía en sus programas de Python.

## Índice

1. Introducción a `asyncio` en Python
2. `async` y `await`: Coroutines y el Event Loop
3. Tareas (`Tasks`) y Programación Concurrente
4. Objetos Compatibles con `asyncio`
5. Profundización en `await`: Cómo la CPU Interviene en I/O-bound
6. Uso de `asyncio.Queue`, `asyncio.Streams`, y `asyncio.Timeouts`


## 1. Introducción a `asyncio` en Python

El módulo `asyncio` de Python permite escribir código concurrente usando la sintaxis `async` y `await`. La asincronía en Python permite gestionar múltiples tareas que se intercalan, lo que resulta útil para operaciones de I/O que podrían bloquear el flujo de un programa.

El `asyncio` se diseñó para resolver problemas relacionados con operaciones I/O-bound. Estas son tareas en las que la CPU pasa la mayor parte del tiempo esperando respuestas de otras operaciones, como solicitudes de red, consultas a bases de datos, lectura/escritura en disco, etc. Cuando el tiempo de espera es significativo, la asincronía permite gestionar eficientemente las tareas.

Las tareas **CPU-bound**, por otro lado, implican operaciones intensivas para la CPU (como cálculos complejos o procesamiento de imágenes) y generalmente no se benefician de `asyncio`. Para estas, otras técnicas como `multiprocessing` o subprocesos pueden ser más adecuadas.

## 2. `async` y `await`: Coroutines y el Event Loop

`asyncio` permite definir coroutines, que son funciones asíncronas que se pueden pausar y reanudar en puntos específicos de su ejecución. Esto se logra con la sintaxis `async def` para definir coroutines y `await` para pausar y reanudar la ejecución.

### Coroutines

Una coroutine es una función especial que devuelve un "coroutine object" cuando se llama. Esto es similar a las funciones generadoras que devuelven un objeto generador.

```python
import asyncio

async def say_hello():
    print("Hello")
    await asyncio.sleep(1)  # Espera un segundo sin bloquear el event loop
    print("World")

asyncio.run(say_hello())
```

En este ejemplo, `say_hello` es una coroutine que se suspende por 1 segundo con `await asyncio.sleep(1)`. Durante este tiempo de espera, el event loop puede ejecutar otras tareas.

### Event Loop

El event loop es el núcleo de `asyncio`. Su trabajo es ejecutar tareas asíncronas de manera concurrente. Al llamar a `asyncio.run(...)`, se inicia el event loop que se encargará de manejar todas las coroutines registradas.

```python
async def say_hello():
    print("Hello")
    await asyncio.sleep(1)
    print("World")

# Correr el event loop
asyncio.run(say_hello())
```

En este caso, el event loop ejecuta la coroutine `say_hello`, suspendiéndola durante 1 segundo y luego continuando su ejecución.

### `await`: Pausar y Reanudar

La palabra clave `await` se usa para pausar la ejecución de la coroutine actual hasta que la operación asíncrona esperada se complete. Esto libera el control para que el event loop pueda ejecutar otras tareas pendientes.

#### Ejemplo con Varias Tareas

```python
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    task1 = asyncio.create_task(say_after(1, 'Hello'))
    task2 = asyncio.create_task(say_after(2, 'World'))

    print("Waiting for tasks...")
    await task1  # Espera que task1 complete
    await task2  # Espera que task2 complete

asyncio.run(main())
```

En este ejemplo, las dos tareas (`task1` y `task2`) se ejecutan de forma concurrente. La salida es:

```
Waiting for tasks...
Hello
World
```

Las tareas se ejecutan de forma concurrente, y el tiempo de espera para cada una es manejado por el event loop.

### Desempaquetado con `*` y `await asyncio.gather`

Para ejecutar múltiples coroutines al mismo tiempo y esperar a que todas se completen, se puede utilizar `asyncio.gather`. La sintaxis `*` es útil para pasar una lista de tareas como argumentos separados a `gather`.

```python
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    # Definir una lista de tareas
    tasks = [
        say_after(1, 'First'),
        say_after(2, 'Second'),
        say_after(3, 'Third')
    ]
    
    # Desempaquetar la lista usando * y pasarla a gather
    await asyncio.gather(*tasks)

asyncio.run(main())
```

En este caso, las tres tareas se ejecutan concurrentemente y el programa espera a que todas completen. La salida será:

```
First
Second
Third
```


## 3. Creación de Objetos Compatibles con `asyncio`

Es posible construir funciones y clases que sean compatibles con `asyncio`. Esto implica utilizar `async` para definir coroutines y `await` para operaciones asíncronas. Además, se pueden crear contextos asíncronos (`async with`) para gestionar recursos de manera segura.

### Definir Métodos Asíncronos en Clases

Para que una clase sea compatible con `asyncio`, simplemente define métodos usando `async def`.

```python
class AsyncWorker:
    async def work(self, delay):
        print("Starting work...")
        await asyncio.sleep(delay)
        print("Finished work!")

async def main():
    worker = AsyncWorker()
    await worker.work(2)

asyncio.run(main())
```

En este ejemplo, se crea una clase `AsyncWorker` con un método asíncrono `work`. El uso de `await` permite suspender la ejecución y liberar la CPU durante la espera.

### Uso de `async with` para Contextos Asíncronos

Es posible usar `async with` para trabajar con recursos asíncronos que necesiten ser gestionados correctamente, como conexiones de red o archivos. Para ello, se definen los métodos `__aenter__` y `__aexit__` en la clase.

```python
class AsyncConnection:
    async def __aenter__(self):
        print("Opening connection...")
        await asyncio.sleep(1)
        print("Connection opened.")
        return self

    async def __aexit__(self, exc_type, exc, tb):
        print("Closing connection...")
        await asyncio.sleep(1)
        print("Connection closed.")

    async def send(self, data):
        print(f"Sending: {data}")
        await asyncio.sleep(0.5)

async def main():
    async with AsyncConnection() as conn:
        await conn.send("Hello!")

asyncio.run(main())
```

En este caso, `async with` garantiza que los recursos (como la conexión) se abran y cierren correctamente. Esto es especialmente útil para gestionar conexiones de red, archivos, o cualquier otro recurso asíncrono que necesite limpieza al finalizar.

## 4. I/O-bound vs CPU-bound: Diferencias Fundamentales

Es importante comprender la diferencia entre tareas **I/O-bound** y **CPU-bound** para saber cuándo `asyncio` es la mejor solución.

### Operaciones I/O-bound

Las tareas I/O-bound son aquellas en las que la mayor parte del tiempo se pasa esperando operaciones de entrada/salida (como acceso a discos, llamadas de red, y comunicación con bases de datos remotas). Estas tareas son ideales para `asyncio` ya que permiten a la CPU liberar recursos mientras espera que el dispositivo I/O complete su operación.

### Operaciones CPU-bound

Las tareas CPU-bound involucran cálculos intensivos que requieren procesamiento activo de la CPU (por ejemplo, procesamiento de imágenes, cálculos complejos, etc.). En estos casos, `asyncio` no es tan efectivo ya que la CPU está ocupada ejecutando la operación, y no hay beneficios en suspenderla. En su lugar, se recomiendan enfoques como `multiprocessing` o el uso de subprocesos.


### ¿Por qué Operaciones de Disco y `sleep` Son I/O-bound?

#### Lectura y Escritura de Archivos

Cuando se lee o escribe en el disco, la operación la maneja el controlador del dispositivo de almacenamiento (por ejemplo, HDD o SSD). La CPU solo inicia la operación y luego espera a que el controlador complete la tarea. La mayor parte del tiempo la CPU está inactiva, esperando que el dispositivo de almacenamiento termine su operación.

#### `asyncio.sleep()` y Temporizador del Sistema

`asyncio.sleep()` no bloquea la CPU. Cuando se llama a `await asyncio.sleep(n)`, `asyncio` notifica al sistema operativo para que reanude la ejecución después de `n` segundos. Esta operación es manejada por el temporizador del sistema operativo y el kernel, liberando la CPU hasta que el tiempo de espera se complete.

```python
import asyncio

async def example():
    print("Start waiting...")
    await asyncio.sleep(2)
    print("Done waiting!")

asyncio.run(example())
```

En este ejemplo, `await asyncio.sleep(2)` suspende la ejecución por 2 segundos, permitiendo que el event loop continúe ejecutando otras tareas.

## 5. Uso de `asyncio.Queue`, `asyncio.Streams`, y `asyncio.Timeouts`

Estas son funcionalidades importantes de `asyncio` para manejar comunicación entre tareas, trabajar con flujos de datos (streams), y gestionar tiempos de espera.


### `asyncio.Queue`: Comunicación entre Tareas

`asyncio.Queue` es una cola segura para múltiples productores y consumidores en `asyncio`. Permite pasar datos de manera segura entre diferentes coroutines de un event loop.

```python
async def producer(queue):
    for i in range(5):
        print(f'Producing item {i}')
        await asyncio.sleep(1)  # Simula una operación de producción
        await queue.put(i)  # Añade el item a la cola
    await queue.put(None)  # Señal para detener a los consumidores

async def consumer(queue):
    while True:
        item = await queue.get()  # Obtiene un item de la cola
        if item is None:
            break  # Si se recibe None, la producción ha terminado
        print(f'Consuming item {item}')
        await asyncio.sleep(2)  # Simula una operación de consumo
    print('Consumer done.')

async def main():
    queue = asyncio.Queue()  # Crea una cola
    await asyncio.gather(producer(queue), consumer(queue))

asyncio.run(main())
```

En este ejemplo, `producer` genera items y los coloca en una cola, mientras que `consumer` los consume uno por uno. La cola facilita la comunicación entre tareas de manera segura.

### `asyncio.Streams`: Trabajar con Conexiones de Red

`asyncio.Streams` facilita la lectura y escritura de datos desde y hacia streams, como conexiones de red (sockets). La combinación de `StreamReader` y `StreamWriter` permite trabajar con conexiones TCP.

#### Crear un Servidor TCP Asíncrono

```python
async def handle_client(reader, writer):
    data = await reader.read(100)  # Lee hasta 100 bytes
    message = data.decode()
    addr = writer.get_extra_info('peername')

    print(f"Received {message} from {addr}")

    writer.write(b"Hello back")
    await writer.drain()  # Espera hasta que el buffer se vacíe

    print("Closing connection")
    writer.close()
    await writer.wait_closed()

async def main():
    server = await asyncio.start_server(handle_client, '127.0.0.1', 8888)
    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

    async with server:
        await server.serve_forever()

asyncio.run(main())
```

Este servidor TCP acepta conexiones en la dirección `127.0.0.1:8888`, lee mensajes de los clientes y envía respuestas. Los `StreamReader` y `StreamWriter` se pasan automáticamente a la coroutine `handle_client` cada vez que un cliente se conecta.

### `asyncio.Timeouts`: Gestionar Tiempos de Espera

Las operaciones pueden tener límites de tiempo usando `asyncio.wait_for`. Si la operación tarda más de lo especificado, se lanza una excepción `asyncio.TimeoutError`.

```python
async def long_task():
    print('Starting task...')
    await asyncio.sleep(5)
    print('Task complete!')

async def main():
    try:
        # Establece un tiempo máximo de espera de 3 segundos para `long_task()`
        await asyncio.wait_for(long_task(), timeout=3)
    except asyncio.TimeoutError:
        print('The task timed out!')

asyncio.run(main())
```

En este ejemplo, la tarea `long_task()` tarda 5 segundos, pero el tiempo máximo de espera es de 3 segundos. Esto genera una excepción `asyncio.TimeoutError` y se imprime el mensaje de tiempo de espera.